## The Battle of the Neighborhoods - Week 2


#### Part 2

### Scraping Population data of NYC 
WEBLINK : https://en.wikipedia.org/wiki/New_York_City

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker

# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y
import seaborn as sns

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c anaconda beautifulsoup4 --yes # package for parsing HTML and XML documents
from bs4 import BeautifulSoup as bs
import csv
print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [44]:
url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soup = bs(url,"html.parser")
table = soup.find('table',{'class':'wikitable sortable'})

headers = [header.text for header in table.find_all('th')]
table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('nyc_population.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)
 

#### Loading Data from CSV file

In [45]:
population_data=pd.read_csv('nyc_population.csv')
population_data.drop(population_data.columns[[7,8,9,10,11]], axis=1,inplace=True)
print('Data downloaded!')

Data downloaded!


##### Fomatting data renaming columns and removing white spaces 

In [46]:
population_data.columns = population_data.columns.str.replace(' ', '')
population_data.columns = population_data.columns.str.replace('\'','')
population_data.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
population_data

,NewYorkCitysfiveboroughsvte\n,Jurisdiction\n,Population\n,GrossDomesticProduct\n,Landarea\n,Density\n,persons_sq_mi,squarekm,persons/sq.mi,persons/km2\n
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
population_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'Borough',
                   'Jurisdiction\n':'County',
                   'Population\n':'Estimate_2017', 
                   'Landarea\n':'square_miles',
                    'Density\n':'square_km'}, inplace=True)
population_data

,Borough,County,Estimate_2017,GrossDomesticProduct\n,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2\n
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.695\n,"30,100\n",42.10\n,109.04\n,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,559,903\n",91.559\n,"35,800\n",70.82\n,183.42\n,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,628,706\n",600.244\n,"368,500\n",22.83\n,59.13\n,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,253,858\n",93.310\n,"41,400\n",108.53\n,281.09\n,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"476,143\n",14.514\n,"30,500\n",58.37\n,151.18\n,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###### Replace newline('\n') from each string from left and right side

In [48]:
population_data['Borough']=population_data['Borough'].replace(to_replace='\n', value='', regex=True)
population_data['County']=population_data['County'].replace(to_replace='\n', value='', regex=True)
population_data['Estimate_2017']=population_data['Estimate_2017'].replace(to_replace='\n', value='', regex=True)
population_data['square_miles']=population_data['square_miles'].replace(to_replace='\n', value='', regex=True)
population_data['square_km']=population_data['square_km'].replace(to_replace='\n', value='', regex=True)
population_data['persons_sq_mi']=population_data['persons_sq_mi'].replace(to_replace='\n', value='', regex=True)
population_data['squarekm']=population_data['squarekm'].replace(to_replace='\n', value='', regex=True)

population_data

,Borough,County,Estimate_2017,GrossDomesticProduct\n,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2\n
0,The Bronx,Bronx,"1,418,207",42.695\n,"30,100",42.10,109.04,NaN,NaN,NaN
1,Brooklyn,Kings,"2,559,903",91.559\n,"35,800",70.82,183.42,NaN,NaN,NaN
2,Manhattan,New York,"1,628,706",600.244\n,"368,500",22.83,59.13,NaN,NaN,NaN
3,Queens,Queens,"2,253,858",93.310\n,"41,400",108.53,281.09,NaN,NaN,NaN
4,Staten Island,Richmond,"476,143",14.514\n,"30,500",58.37,151.18,NaN,NaN,NaN
5,City of New York,"8,336,817",842.343,"101,000",302.64,783.83,"27,547",NaN,NaN,NaN
6,State of New York,"19,453,561","1,731.910","89,000","47,214","122,284",412,NaN,NaN,NaN
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Last 3 Rows are not Boroughs we we have to shift the data as part of data formatting

In [49]:
population_data.loc[5:,['persons_sq_mi','squarekm']] = population_data.loc[2:,['persons_sq_mi','squarekm']].shift(1,axis=1)
population_data.loc[5:,['square_km','persons_sq_mi']] = population_data.loc[2:,['square_km','persons_sq_mi']].shift(1,axis=1)
population_data.loc[5:,['square_miles','square_km']] = population_data.loc[2:,['square_miles','square_km']].shift(1,axis=1)
population_data.loc[5:,['Estimate_2017','square_miles']] = population_data.loc[2:,['Estimate_2017','square_miles']].shift(1,axis=1)
population_data.loc[5:,['County','Estimate_2017']] = population_data.loc[2:,['County','Estimate_2017']].shift(1,axis=1)
population_data.loc[5:,['Borough','County']] = population_data.loc[2:,['Borough','County']].shift(1,axis=1)
population_data

,Borough,County,Estimate_2017,GrossDomesticProduct\n,square_miles,square_km,persons_sq_mi,squarekm,persons/sq.mi,persons/km2\n
0,The Bronx,Bronx,"1,418,207",42.695\n,"30,100",42.10,109.04,NaN,NaN,NaN
1,Brooklyn,Kings,"2,559,903",91.559\n,"35,800",70.82,183.42,NaN,NaN,NaN
2,Manhattan,New York,"1,628,706",600.244\n,"368,500",22.83,59.13,NaN,NaN,NaN
3,Queens,Queens,"2,253,858",93.310\n,"41,400",108.53,281.09,NaN,NaN,NaN
4,Staten Island,Richmond,"476,143",14.514\n,"30,500",58.37,151.18,NaN,NaN,NaN
5,NaN,City of New York,"8,336,817","101,000",842.343,302.64,783.83,NaN,NaN,NaN
6,NaN,State of New York,"19,453,561","89,000","1,731.910","47,214","122,284",NaN,NaN,NaN
7,NaN,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Last 2 rows are not required lets remove/drop them

In [50]:
i = population_data[((population_data.County == 'Sources: [2] and see individual borough articles'))].index
population_data.drop(i,inplace=True)
j=population_data[((population_data.County == 'Sources:[14] and see individual borough articles'))].index
population_data.drop(j,inplace=True)

In [51]:
population_data["Estimate_2017"]=population_data["Estimate_2017"].str.replace("," , "").astype(str)
population_data.drop(["squarekm","persons/sq.mi","persons/sq.mi","persons/km2\n"], axis = 1,inplace=True)

In [52]:
population_data['Borough']= population_data['Borough'].str.replace(' The Bronx', 'Bronx')
population_data['Borough']= population_data['Borough'].str.replace(' ', '')
population_data['Borough']= population_data['Borough'].str.replace(',', '')




## Scarping Demographics data from Wikipedia page

We will web scrap Demographics data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City

In [70]:
url = 'https://en.wikipedia.org/wiki/New_York_City'
website_url = requests.get(url).text
soup =bs(website_url,'html.parser')
tables = soup.find_all('table')

for table in tables:
    if 'Racial composition' in table.text:
        headers = [header.text.strip() for header in table.find_all('th')]
        rows = []
        table_rows = table.find_all('tr')    
        for row in table_rows:
           td = row.find_all('td')
           row = [row.text for row in td]
           rows.append(row)

df = pd.DataFrame(rows, columns=headers)  

In [71]:
df

,Racial composition,2010[234],1990[236],1970[236],1940[236]
0,None,None,None,None,None
1,White,44.0%,52.3%,76.6%,93.6%\n
2,—Non-Hispanic,33.3%,43.2%,62.9%[237],92.0%\n
3,Black or African American,25.5%,28.7%,21.1%,6.1%\n
4,Hispanic or Latino (of any race),28.6%,24.4%,16.2%[237],1.6%\n
5,Asian,12.7%,7.0%,1.2%,–\n


#### Renaming Cloumns


In [72]:
df.rename(columns = {'2010[234]' : '2010',
                   '1990[236]':'1990',
                   '1970[236]':'1970', 
                   '1940[236]':'1940',
                    }, inplace=True)
df

,Racial composition,2010,1990,1970,1940
0,None,None,None,None,None
1,White,44.0%,52.3%,76.6%,93.6%\n
2,—Non-Hispanic,33.3%,43.2%,62.9%[237],92.0%\n
3,Black or African American,25.5%,28.7%,21.1%,6.1%\n
4,Hispanic or Latino (of any race),28.6%,24.4%,16.2%[237],1.6%\n
5,Asian,12.7%,7.0%,1.2%,–\n


In [73]:
## Formating data
df.columns = df.columns.str.replace(' ', '')

In [91]:
df= df.replace('\n',' ', regex=True)
df["Racialcomposition"]= df["Racialcomposition"].replace('—Non-Hispanic','Non-Hispanic', regex=True)
#k = df[((df.'2010'== 'None'))].index
#df.drop(k,inplace=True)
df

,Racialcomposition,2010,1990,1970,1940
0,None,None,None,None,None
1,White,44.0%,52.3%,76.6%,93.6%
2,Non-Hispanic,33.3%,43.2%,62.9%,92.0%
3,Black or African American,25.5%,28.7%,21.1%,6.1%
4,Hispanic or Latino (of any race),28.6%,24.4%,16.2%,1.6%
5,Asian,12.7%,7.0%,1.2%,–


### Removing 237 from col 1970

In [92]:
df['1970'] = df['1970'].str.rstrip('[237]')
df

,Racialcomposition,2010,1990,1970,1940
0,None,None,None,None,None
1,White,44.0%,52.3%,76.6%,93.6%
2,Non-Hispanic,33.3%,43.2%,62.9%,92.0%
3,Black or African American,25.5%,28.7%,21.1%,6.1%
4,Hispanic or Latino (of any race),28.6%,24.4%,16.2%,1.6%
5,Asian,12.7%,7.0%,1.2%,–


In [93]:
df.to_csv('nyc-demographics.csv',index=False)